In [1]:
import random
import numpy as np

In [2]:
def getInfo(nonogram):
    return nonogram.get("rows"), \
           nonogram.get("cols"), \
           [clue for row in nonogram.get("rows") for clue in row], \
           [clue for col in nonogram.get("cols") for clue in col]

In [3]:
def defineGenotype(cols):
    genotype = []

    for col in cols:
        genotype.append({
            "col": col,
            "genom": [random.randrange(101) for _ in range(len(col))]
        })

    return genotype

In [4]:
def decodeGenotype(rows, genotype):
    decoded = []

    for elem in genotype:
        col = elem.get("col")
        genom = elem.get("genom")

        decoded_col = [[] for _ in range(len(rows))]

        variables = {
            "start": 0,
            "allowed": None,
            "placed": None
        }

        for idx, clue in enumerate(col):
            variables["allowed"] = 0

            bk_right = (len(decoded_col) - 1) - (sum(col[idx + 1:]) + len(col[idx + 1:]))
            bk_left = int(next((idx + 2 for idx, elem in reversed(list(enumerate(decoded_col))) if elem), '0'))

            for index_place, place in enumerate(decoded_col[bk_left: bk_right + 1]):
                if index_place + bk_left + clue <= bk_right + 1:
                    variables["allowed"] += 1

            variables["placed"] = variables.get("start") + genom[idx] % variables.get("allowed")

            for index_place, place in enumerate(decoded_col[variables.get("placed"): variables.get("placed") + clue]):
                place.append('x')

            variables["start"] = variables.get("placed") + clue + 1

        decoded.append(decoded_col)

    return decoded

In [5]:
def getSolution(cols, decoded_geno):
    solution = [[] for _ in range(len(cols))]
    
    column = 0
    
    for col in decoded_geno:
        count = 0
        
        for index, elem in enumerate(col):
            if index == 0:
                if not elem:
                    count += 1
                else:
                    solution[column].append(count)
                    count -= count
            else:
                if not elem:
                    count += 1
                elif count == 0:
                    pass
                elif next((True for elem in col[:index] if elem), False):
                    solution[column].append(count - 1)
                    count -= count
                else:
                    solution[column].append(count)
                    count -= count
                    
        column += 1

    return solution

In [6]:
def getNonogramFromIndividual(rows, col_blocks, individual):
    decoded_ind = []
    i = 0

    for col in individual:
        decoded_col = [[] for _ in range(len(rows))]

        for idx, block in enumerate(col):
            if idx == 0:
                for place in range(block, block + col_blocks[i]):
                    decoded_col[place].append('x')
            else:
                last_place = next((idx + 2 for idx, elem in reversed(list(enumerate(decoded_col))) if elem))
                for place in range(last_place + block, last_place + block + col_blocks[i]):
                    decoded_col[place].append('x')
            i += 1

        decoded_ind.append(decoded_col)
        
    numpy_array = np.array(decoded_ind, dtype=object)
    transpose = numpy_array.T

    return transpose.tolist()

In [7]:
def getRowsFromNonogram(nonogram):
    to_strings = [''.join(map(lambda s: 'x' if s else ' ', row)) for row in nonogram]
    return [list(map(len, s.split())) for s in to_strings]